In [1]:
# import comet_ml at the top of your file
from comet_ml import Experiment
import comet_ml
import logging
# Create an experiment with your api key
experiment = Experiment(
    project_name="digit-recognizer",
    workspace="wodenwang820118",
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/wodenwang820118/digit-recognizer/37ce9e0096114a6dad158e8aa8ef3683



In [2]:
logging.basicConfig(level=logging.INFO)
LOGGER = logging.getLogger("comet_ml")

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras

%matplotlib inline
sns.set_style('whitegrid')

In [3]:
# tensorflow 2.7 
from tensorflow import keras
from tensorflow.keras.layers import Input,Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers
from tensorflow.keras import activations

In [4]:
# scale the numeric data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [5]:
df_train=pd.read_csv('train.csv')
df_train.head(1)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
y = df_train['label']
X = df_train.drop('label', axis=1)

In [7]:
X = scaler.fit_transform(X)

In [8]:
y.unique()

array([1, 0, 4, 7, 3, 5, 8, 9, 2, 6], dtype=int64)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [10]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((28140, 784), (28140,), (13860, 784), (13860,))

In [11]:
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=3, verbose=1, mode='auto', min_delta=0.00001, cooldown=0, min_lr=0.0001)

In [14]:
class DigitClassifier:
    def __init__(self,X_train,X_test,y_train,y_test,early_stop,reduce_lr,experiment):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.reduce_lr = reduce_lr
        self.early_stop = early_stop
        self.experiment = experiment
    
    def build_model(self):
        # Build the model
        model = Sequential()
        model.add(
            Dense(
                units=self.experiment.get_parameter("first_layer_units"),
                input_shape=(self.X_train.shape[1],)
            )
        )
        model.add(BatchNormalization())
        model.add(layers.Activation(activations.elu))
        model.add(Dropout(self.experiment.get_parameter("first_layer_dropout_rate")))

        model.add(
            Dense(
                units=self.experiment.get_parameter("second_layer_units"),
                )
            )
        model.add(BatchNormalization())
        model.add(layers.Activation(activations.elu))
        model.add(Dropout(self.experiment.get_parameter("second_layer_dropout_rate")))

        model.add(
            Dense(
                units=self.experiment.get_parameter("third_layer_units"),
                )
            )
        model.add(BatchNormalization())
        model.add(layers.Activation(activations.elu))
        model.add(Dropout(self.experiment.get_parameter("third_layer_dropout_rate")))
        
        model.add(Dense(10,activation='softmax'))
        optimizer = keras.optimizers.Adam(0.001)
        model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model
    
    def train_model(self):
        # Train the model
        model = self.build_model()
        model.fit(
            self.X_train,
            self.y_train,
            batch_size=self.experiment.get_parameter("batch_size"),
            epochs=self.experiment.get_parameter("epochs"),
            validation_data=(self.X_test,self.y_test),
            callbacks=[self.early_stop,self.reduce_lr]
        )
        return model
    
    def evaluate_model(self):
        # Evaluate the model
        model = self.train_model()
        score = model.evaluate(self.X_test,self.y_test)
        LOGGER.info(f"{ score }")
    
    def grid_search(self, config_dict):
        opt = comet_ml.Optimizer(config_dict)
        for self.experiment in opt.get_experiments(project_name="digit-recognizer"):
            self.experiment.log_parameters("epochs", 10)

            self.build_model()
            self.train_model()
            self.evaluate_model()
            self.experiment.end()

In [15]:
classifier = DigitClassifier(X_train,X_test,y_train,y_test,early_stop,reduce_lr,experiment)
# mu is the mean number of units, sigma is the standard deviation
classifier.grid_search({
    "algorithm": "bayes",
    "name": "Optimize Music Classification Network",
    "spec": {"maxCombo": 10, "objective": "minimize", "metric": "loss"},
    "parameters": {
        "first_layer_units": {"type": "discrete", "values": [450,500,550,600,650,700,750,800,850,900,950,1000]},
        "first_layer_dropout_rate": {"type": "discrete", "values": [0.4,0.5,0.6,0.7,0.8]},
        "second_layer_units": {"type": "discrete", "values": [300,400,450,500,550,600,650,700,750,800]},
        "second_layer_dropout_rate": {"type": "discrete", "values": [0.1,0.2,0.3,0.4,0.5,0.6]},
        "third_layer_units": {"type": "discrete", "values": [50,75,100,200,300,400,450]},
        "third_layer_dropout_rate": {"type": "discrete", "values": [0.1,0.2,0.3,0.4]},
        "batch_size": {"type": "discrete", "values": [32,64]},
        "epochs": {"type": "discrete", "values": [150]},
    },
    "trials": 1,
})

COMET INFO: COMET_OPTIMIZER_ID=d222a9ceb83e44b695c6f48cd870af98
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 201600, 'endTime': None, 'id': 'd222a9ceb83e44b695c6f48cd870af98', 'lastUpdateTime': None, 'maxCombo': 10, 'name': 'Optimize Music Classification Network', 'parameters': {'batch_size': {'type': 'discrete', 'values': [32, 64]}, 'epochs': {'type': 'discrete', 'values': [150]}, 'first_layer_dropout_rate': {'type': 'discrete', 'values': [0.4, 0.5, 0.6, 0.7, 0.8]}, 'first_layer_units': {'type': 'discrete', 'values': [450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000]}, 'second_layer_dropout_rate': {'type': 'discrete', 'values': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]}, 'second_layer_units': {'type': 'discrete', 'values': [300, 400, 450, 500, 550, 600, 650, 700, 750, 800]}, 'third_layer_dropout_rate': {'type': 'discrete', 'values': [0.1, 0.2, 0.3, 0.4]}, 'third_layer_units': {'type': 'discrete', 'values': [50, 75, 100, 200, 300, 400, 450]}}, 'predicto

Epoch 1/150


COMET INFO: ignoring tensorflow summary log of metrics because of keras; set `comet_ml.loggers.tensorboard_logger.LOG_METRICS = True` to override


880/880 [==============================] - 15s 14ms/step - loss: 0.4412 - accuracy: 0.8647 - val_loss: 0.2370 - val_accuracy: 0.9293 - lr: 0.0010
Epoch 2/150
880/880 [==============================] - 9s 10ms/step - loss: 0.3037 - accuracy: 0.9071 - val_loss: 0.1983 - val_accuracy: 0.9398 - lr: 0.0010
Epoch 3/150
880/880 [==============================] - 4s 5ms/step - loss: 0.2559 - accuracy: 0.9211 - val_loss: 0.1824 - val_accuracy: 0.9458 - lr: 0.0010
Epoch 4/150
880/880 [==============================] - 7s 8ms/step - loss: 0.2268 - accuracy: 0.9291 - val_loss: 0.1698 - val_accuracy: 0.9489 - lr: 0.0010
Epoch 5/150
880/880 [==============================] - 4s 4ms/step - loss: 0.2046 - accuracy: 0.9346 - val_loss: 0.1513 - val_accuracy: 0.9548 - lr: 0.0010
Epoch 6/150
880/880 [==============================] - 3s 4ms/step - loss: 0.1870 - accuracy: 0.9399 - val_loss: 0.1468 - val_accuracy: 0.9543 - lr: 0.0010
Epoch 7/150
880/880 [==============================] - 5s 6ms/step - loss

COMET INFO: [0.09876728057861328, 0.9753968119621277]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/digit-recognizer/a0bee2ad8b5a4d21980761c1ef845710
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [97]                  : (0.8639658689498901, 0.9900497794151306)
COMET INFO:     batch_accuracy [8536]          : (0.03125, 1.0)
COMET INFO:     batch_loss [8536]              : (0.00022866768995299935, 2.8641672134399414)
COMET INFO:     epoch_duration [97]            : (3.42200000000048, 14.718999999999141)
COMET INFO:     loss [97]                      : (0.02974042482674122, 0.4416247010231018)
COMET INFO:     lr [97]                        : (9.999999747378752e-05, 0.0010000000474974513)
COMET INFO:     val_accuracy [97]              : (0.9182539582252502, 0.

...

880/880 [==============================] - 4s 4ms/step - loss: 0.6122 - accuracy: 0.8083 - val_loss: 0.2833 - val_accuracy: 0.9143 - lr: 0.0010
Epoch 2/150
880/880 [==============================] - 3s 4ms/step - loss: 0.4142 - accuracy: 0.8726 - val_loss: 0.2579 - val_accuracy: 0.9193 - lr: 0.0010
Epoch 3/150
880/880 [==============================] - 4s 4ms/step - loss: 0.3673 - accuracy: 0.8869 - val_loss: 0.2345 - val_accuracy: 0.9280 - lr: 0.0010
Epoch 4/150
880/880 [==============================] - 3s 4ms/step - loss: 0.3403 - accuracy: 0.8962 - val_loss: 0.2058 - val_accuracy: 0.9351 - lr: 0.0010
Epoch 5/150
880/880 [==============================] - 3s 4ms/step - loss: 0.3115 - accuracy: 0.9025 - val_loss: 0.1950 - val_accuracy: 0.9382 - lr: 0.0010
Epoch 6/150
880/880 [==============================] - 3s 4ms/step - loss: 0.3010 - accuracy: 0.9064 - val_loss: 0.1919 - val_accuracy: 0.9412 - lr: 0.0010
Epoch 7/150
880/880 [==============================] - 3s 4ms/step - loss: 0

COMET INFO: [0.09570042043924332, 0.972582995891571]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/digit-recognizer/5fe16cefda15411fa78984a9e6e4d160
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [205]                 : (0.8082800507545471, 0.9719971418380737)
COMET INFO:     batch_accuracy [18040]         : (0.0625, 1.0)
COMET INFO:     batch_loss [18040]             : (0.005454329773783684, 2.944427490234375)
COMET INFO:     epoch_duration [205]           : (3.375, 4.21900000000096)
COMET INFO:     loss [205]                     : (0.08804254978895187, 0.612510085105896)
COMET INFO:     lr [205]                       : (9.999999747378752e-05, 0.0010000000474974513)
COMET INFO:     val_accuracy [205]             : (0.91118323802948, 0.9735209345817566)
COM

Epoch 1/150
  1/880 [..............................] - ETA: 6:51 - loss: 2.6059 - accuracy: 0.0625

880/880 [==============================] - 4s 4ms/step - loss: 0.4623 - accuracy: 0.8613 - val_loss: 0.2556 - val_accuracy: 0.9227 - lr: 0.0010
Epoch 2/150
880/880 [==============================] - 3s 4ms/step - loss: 0.3020 - accuracy: 0.9096 - val_loss: 0.2017 - val_accuracy: 0.9386 - lr: 0.0010
Epoch 3/150
880/880 [==============================] - 3s 4ms/step - loss: 0.2477 - accuracy: 0.9256 - val_loss: 0.1764 - val_accuracy: 0.9457 - lr: 0.0010
Epoch 4/150
880/880 [==============================] - 3s 4ms/step - loss: 0.2172 - accuracy: 0.9333 - val_loss: 0.1547 - val_accuracy: 0.9522 - lr: 0.0010
Epoch 5/150
880/880 [==============================] - 3s 4ms/step - loss: 0.1884 - accuracy: 0.9417 - val_loss: 0.1414 - val_accuracy: 0.9563 - lr: 0.0010
Epoch 6/150
880/880 [==============================] - 3s 4ms/step - loss: 0.1737 - accuracy: 0.9465 - val_loss: 0.1317 - val_accuracy: 0.9592 - lr: 0.0010
Epoch 7/150
880/880 [==============================] - 3s 4ms/step - loss: 0

COMET INFO: [0.09699351340532303, 0.9769119620323181]
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/wodenwang820118/digit-recognizer/f29fa281ee454536b2e3fb8590345a2e
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [75]                  : (0.8612651228904724, 0.9932480454444885)
COMET INFO:     batch_accuracy [6600]          : (0.0625, 1.0)
COMET INFO:     batch_loss [6600]              : (0.001891818828880787, 3.096632719039917)
COMET INFO:     epoch_duration [75]            : (3.375, 4.359000000000378)
COMET INFO:     loss [75]                      : (0.021459097042679787, 0.4622652530670166)
COMET INFO:     lr [75]                        : (0.00011764899682020769, 0.0010000000474974513)
COMET INFO:     val_accuracy [75]              : (0.9227272868156433, 0.97799420356750

In [13]:
# Build the model
model = Sequential()
model.add(
    Dense(
        units=950,
        input_shape=(X_train.shape[1],)
    )
)
model.add(BatchNormalization())
model.add(layers.Activation(activations.elu))
model.add(Dropout(0.7))

model.add(
    Dense(
        units=600,
        )
    )
model.add(BatchNormalization())
model.add(layers.Activation(activations.elu))
model.add(Dropout(0.1))

model.add(
    Dense(
        units=400,
        )
    )
model.add(BatchNormalization())
model.add(layers.Activation(activations.elu))
model.add(Dropout(0.1))

model.add(Dense(10,activation='softmax'))
optimizer = keras.optimizers.Adam(0.001)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
model.fit(X_train,y_train,batch_size=32,epochs=200,validation_data=(X_test,y_test),callbacks=[early_stop,reduce_lr])

Epoch 1/200
880/880 [==============================] - 5s 5ms/step - loss: 0.4705 - accuracy: 0.8566 - val_loss: 0.2434 - val_accuracy: 0.9234 - lr: 0.0010
Epoch 2/200
880/880 [==============================] - 4s 5ms/step - loss: 0.3037 - accuracy: 0.9061 - val_loss: 0.1978 - val_accuracy: 0.9390 - lr: 0.0010
Epoch 3/200
880/880 [==============================] - 4s 5ms/step - loss: 0.2571 - accuracy: 0.9200 - val_loss: 0.1717 - val_accuracy: 0.9466 - lr: 0.0010
Epoch 4/200
880/880 [==============================] - 4s 5ms/step - loss: 0.2385 - accuracy: 0.9237 - val_loss: 0.1658 - val_accuracy: 0.9481 - lr: 0.0010
Epoch 5/200
880/880 [==============================] - 4s 4ms/step - loss: 0.2179 - accuracy: 0.9311 - val_loss: 0.1528 - val_accuracy: 0.9522 - lr: 0.0010
Epoch 6/200
880/880 [==============================] - 4s 5ms/step - loss: 0.2029 - accuracy: 0.9349 - val_loss: 0.1435 - val_accuracy: 0.9578 - lr: 0.0010
Epoch 7/200
880/880 [==============================] - 6s 6ms/st

In [22]:
model.evaluate(X_test,y_test)

434/434 [==============================] - 1s 2ms/step - loss: 0.0832 - accuracy: 0.9773


[0.08322234451770782, 0.9773448705673218]

In [23]:
df_test = pd.read_csv('test.csv')
# scale the numeric data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
test_data = scaler.fit_transform(df_test)

In [24]:
prediction = model.predict(test_data)

In [25]:
labels = np.argmax(prediction, axis=1)

In [26]:
df_result = pd.DataFrame(np.arange(1,len(labels)+1), columns=['ImageId'])
df_result['label']=labels

In [27]:
df_result.head()

,ImageId,label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [28]:
df_result.to_csv('submission_nn_comet_keras.csv', index=False)